In [29]:
import pandas as pd
import numpy as np
import plotly.graph_objs as go

# Read in the data from the Excel sheet
df = pd.read_excel('input.xlsx')

# Convert the observation_date column to a pandas datetime object
df['observation_date'] = pd.to_datetime(df['observation_date'])

# Sort the data by observation date
df = df.sort_values('observation_date')

# Define the number of simulations and periods
num_simulations = 1000000
# Days
num_periods = 90

def monte_carlo_sim(df, num_simulations, num_periods):
    # Get the most recent value
    last_value = df.iloc[-1]['DCOILWTICO']
    
    # Calculate the daily percent change in the value
    daily_returns = df['DCOILWTICO'].pct_change()
    daily_returns = daily_returns.dropna()
    
    # Calculate the mean and standard deviation of the daily returns
    mean_return = daily_returns.mean()
    std_return = daily_returns.std()
    
    # Generate the random daily returns for the simulations
    returns = np.random.normal(mean_return, std_return, (num_simulations, num_periods))
    
    # Calculate the future values for each simulation
    future_values = np.zeros((num_simulations, num_periods))
    future_values[:, 0] = last_value * (1 + returns[:, 0])
    for i in range(1, num_periods):
        future_values[:, i] = future_values[:, i-1] * (1 + returns[:, i])
    
    # Calculate the mean, lower 95%, and upper 95% predictions
    mean_prediction = np.mean(future_values, axis=0)
    lower_prediction_95 = np.percentile(future_values, 5, axis=0)
    upper_prediction_95 = np.percentile(future_values, 95, axis=0)
    lower_prediction_75 = np.percentile(future_values, 25, axis=0)
    upper_prediction_75 = np.percentile(future_values, 75, axis=0)
    
    return mean_prediction, lower_prediction_95, upper_prediction_95, lower_prediction_75, upper_prediction_75

def plot_forecast(df, mean_prediction, lower_prediction_95, upper_prediction_95, lower_prediction_75, upper_prediction_75):
    # Extract the start and end dates from the data
    start_date = df['observation_date'].min()
    end_date = df['observation_date'].max()
    actual_trace = go.Scatter(x=df['observation_date'], y=df['DCOILWTICO'], name='Actual Data', mode='lines')
    # Create the mean prediction trace
    mean_trace = go.Scatter(x=pd.date_range(start=end_date, periods=num_periods, freq='D'), 
                            y=mean_prediction, name='Mean Prediction', mode='lines')
    
    # Create the lower prediction trace 5th percentile
    lower_trace_95 = go.Scatter(x=pd.date_range(start=end_date, periods=num_periods, freq='D'), 
                             y=lower_prediction_95, name='Lower Prediction 95%', mode='lines', line=dict(dash='dash'))
    
    # Create the upper prediction trace 95th percentile
    upper_trace_95 = go.Scatter(x=pd.date_range(start=end_date, periods=num_periods, freq='D'), 
                             y=upper_prediction_95, name='Upper Prediction 95%', mode='lines', line=dict(dash='dash'))
        # Create the lower prediction trace 25th percentile
    lower_trace_75 = go.Scatter(x=pd.date_range(start=end_date, periods=num_periods, freq='D'), 
                             y=lower_prediction_75, name='Lower Prediction 75%', mode='lines', line=dict(dash='dash'))
    
    # Create the upper prediction trace 75th percentile
    upper_trace_75 = go.Scatter(x=pd.date_range(start=end_date, periods=num_periods, freq='D'), 
                             y=upper_prediction_75, name='Upper Prediction 75%', mode='lines', line=dict(dash='dash'))
    
    # Create the plot
    fig = go.Figure([actual_trace, mean_trace, lower_trace_95, upper_trace_95, lower_trace_75, upper_trace_75])
    
    # Set the plot title and axis labels
    fig.update_layout(title=f'Crude Oil Prices: West Texas Intermediate - Cushing, Oklahoma, Dollars per Barrel: ' + str(num_periods) + ' Day Forecast', xaxis_title='Time', yaxis_title='Daily Price of crude oil (US dollars)')
    
    # Show the plot
    fig.show()


try:
  # Perform the Monte Carlo simulation
  mean_prediction, lower_prediction_95, upper_prediction_95, lower_prediction_75,upper_prediction_75 = monte_carlo_sim(df, num_simulations, num_periods)
      
  # Generate the plot
  plot_forecast(df, mean_prediction, lower_prediction_95, upper_prediction_95, lower_prediction_75,upper_prediction_75)
except:
  print('Error')